![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.6.Light_Deidentification.ipynb)

# Light Deidentification

# Colab setups

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1  spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import json
import os

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

import sparknlp
import sparknlp_jsl

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(secret = license_keys["SECRET"], params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.3.1
Spark NLP_JSL Version : 5.3.1


# LightDeIdentification

Light DeIdentification is designed to accelarete deidentification by removing regex usage, token usage in order to increase performance significantly.

Deidentification process is taking effect after entities recognition. With Light DeIdentification Annotator, we dont interference ner process with regex etc. anymore. So if ner results are satisfactory, then it is recommended to use Light DeIdentification Annotator.

When Defaults parameters are used, Light DeIdentification is approximately faster  4x more than DeIdentification.

Light DeIdentification is a light version of `DeIdentification`. It replaces sensitive information in a text with `obfuscated` or `masked` fakers. It is designed to work with healthcare data, and it can be used to de-identify **patient names, dates**, and other sensitive information. It can also be used to **obfuscate** or **mask** any other type of sensitive information, such as *doctor names, hospital names*, and other types of sensitive informatio Additionally, it supports millions of embedded `fakers` and If desired, `custom external fakers` can be set with **setCustomFakers** function. It also supports multiple languages such as English, Spanish, French, German, and Arabic. And it supports multi-mode de-Identification with **setSelectiveObfuscationModes** function at the same time.



**Alternative from ChatGPT:**

Light DeIdentification is meticulously crafted to revolutionize the deidentification process, eschewing traditional regex and token methods to bolster efficiency and performance manifold.

Post-entity recognition, the DeIdentification process seamlessly integrates with Light DeIdentification Annotator, eliminating any reliance on intrusive regex procedures. Hence, if the NER (Named Entity Recognition) outcomes meet expectations, the employment of Light DeIdentification Annotator is highly recommended.

When employing default parameters, Light DeIdentification showcases a remarkable speed enhancement, boasting approximately fourfold acceleration compared to conventional DeIdentification methods.

As a streamlined variant of DeIdentification, Light DeIdentification excels in substituting sensitive textual elements with meticulously crafted obfuscated or masked placeholders. Primarily tailored for healthcare data, it proficiently anonymizes patient names, dates, and other confidential details. Moreover, it extends its functionality to safeguarding a myriad of sensitive information such as doctor names, hospital identities, and more. Notably, it boasts support for a vast array of embedded fakers, numbering in the millions, with the option to seamlessly integrate custom external fakers using the `setCustomFakers` function.

Furthermore, its versatility shines through with comprehensive language support, encompassing English, Spanish, French, German, and Arabic. Additionally, it offers the flexibility of multi-mode de-Identification through the `setSelectiveObfuscationModes` function, enabling simultaneous application of various obfuscation techniques tailored to specific requirements.

# pipeline


# mode: obfuscate, parameters: defaults
by default, the sensitive data are obfuscated with fake data from JSL Faker, except DATE's

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_subentity")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner_subentity"])\
    .setOutputCol("ner_chunk")

light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setLanguage("en") \
    .setSeed(10) \

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        ner_subentity,
        ner_converter,
        light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [ ]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .
'''

In [ ]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [ ]:
print("detected sensitive data by NER model:\n\n")
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

detected sensitive data by NER model:


+-----------------------------+-------------+
|chunk                        |ner_label    |
+-----------------------------+-------------+
|2093-01-13                   |DATE         |
|David Hale                   |DOCTOR       |
|Hendrickson Ora              |PATIENT      |
|7194334                      |MEDICALRECORD|
|01/13/93                     |DATE         |
|Oliveira                     |DOCTOR       |
|55                           |AGE          |
|2079-11-09                   |DATE         |
|Cocke County Baptist Hospital|HOSPITAL     |
|0295 Keats Street            |STREET       |
|55-555-5555                  |PHONE        |
+-----------------------------+-------------+



In [ ]:
result.select(F.explode(F.arrays_zip(result.sentence.result,
                                     result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()

,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , Harlene Ramus , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Aleda Grana , MR # 1610960 Date : <DATE> ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : Arvil Persons , 53 years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","BOULDER COMMUNITY HOSPITAL , 3020 West Wheatland Road , Phone 45-409-8119 ."


by default parameters, in **obfuscate** mode, recognized entities, except DATE, are obfuscated with values of **Faker** module.

## obfuscate DATEs

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setObfuscateDate(True)\
    .setLanguage("en") \
    .setSeed(10) \


nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        ner_subentity,
        ner_converter,
        light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()


,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-02-20 , Harlene Ramus , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Aleda Grana , MR # 1610960 Date : 02/20/93 ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : Arvil Persons , 53 years-old , Record date : 2079-12-17 ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","BOULDER COMMUNITY HOSPITAL , 3020 West Wheatland Road , Phone 45-409-8119 ."


now `<DATE>` `s in sentence 1 and 2   are also *obfuscated* with fake date from JSL Faker

## Obfuscate with custom fake data

Instead of fake values coming from JSL Faker module, we can use custom fake data

note: when using custom fake data, no need to set these entities names: PHONE, FAX, ID, IDNUM, BIOID, MEDICALRECORD, ZIP, VIN, SSN, DLN, LICENSE, PLATE.

Those entities are obfuscated from JSL Faker module by default.
But if we add `.setSameLengthFormattedEntities([])` parameter, it will take effect.

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setObfuscateDate(True)\
    .setDateFormats(["MM/dd/yyyy","yyyy-MM-dd" ]) \
    .setDays(7) \
    .setObfuscateRefSource('custom') \
    .setCustomFakers({"Doctor": ["John", "Joe"],
                      "Patient": ["James", "Michael"],
                      "Hospital": ["Medical Center"],
                      "Street" : ["Main Street"],
                      "Age":["1","10", "20", "40","80"],
                      "PHONE":["000-000-0000"]
                      }) \
    .setSameLengthFormattedEntities([])\
    .setAgeRanges([1, 4, 12, 20, 40, 60, 80])\
    .setLanguage("en") \
    .setSeed(42) \
    .setDateEntities(["DATE", "DOB",  "DOD"]) \



nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        ner_subentity,
        ner_converter,
        light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555.
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()

,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-20 , John , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Michael , MR # <MEDICALRECORD> Date : 19-08-1941 ."
2,"PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .","PCP : Joe , 95 years-old , Record date : 2079-11-16 ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555.","Medical Center , Main Street , Phone 000-000-0000."


NOTE:  Age over 95 are not obfuscated and date   `01/13/93` is recognized  and obfuscated by JSL Faker, not by .setDays(7), because format is MM/dd/yy and it is not in list of format  `.setDateFormats(["MM/dd/yyyy","yyyy-MM-dd" ]) `


In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setObfuscateDate(True)\
    .setDateFormats(["MM/dd/yyyy","yyyy-MM-dd", "MM/dd/yy"]) \
    .setDays(7) \
    .setObfuscateRefSource('custom') \
    .setCustomFakers({"Doctor": ["John", "Joe"],
                      "Patient": ["James", "Michael"],
                      "Hospital": ["Medical Center"],
                      "Street" : ["Main Street"],
                      "Age":["1","10", "20", "40","80"],"PHONE":["555-555-0000"]
                      }) \
    .setAgeRanges([1, 4, 12, 20, 40, 60, 80])\
    .setLanguage("en") \
    .setSeed(42) \
    .setDateEntities(["DATE", "DOB",  "DOD"]) \



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555.
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()


,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-20 , John , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Michael , MR # 1610960 Date : 01/20/93 ."
2,"PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .","PCP : Joe , 95 years-old , Record date : 2079-11-16 ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555.","Medical Center , Main Street , Phone 45-409-8119."


Now, it is obfuscated as  01/20/93

OR by setting Region with `.setRegion('us') predescribed date formats will take effect

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setObfuscateDate(True)\
    .setDays(7) \
    .setObfuscateRefSource('custom') \
    .setCustomFakers({"Doctor": ["John", "Joe"],
                      "Patient": ["James", "Michael"],
                      "Hospital": ["Medical Center"],
                      "Street" : ["Main Street"],"Phone":["555-555-0000"],
                      "Age":["1","10", "20", "40","80"],"PHONE":["555-555-0000"], "SSN":["123-22-9999"]
                      }) \
    .setAgeRanges([1, 4, 12, 20, 40, 60, 80])\
    .setLanguage("en") \
    .setRegion('us') \
    .setSeed(42) \
    .setDateEntities(["DATE", "DOB",  "DOD"]) \
    .setSameLengthFormattedEntities([])



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 SSN: 123-22-4567.
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()


,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-20 , John , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Michael , MR # <MEDICALRECORD> Date : 01/20/93 ."
2,"PCP : Oliveira , 95 years-old , Record date : 2079-11-09 .","PCP : Joe , 95 years-old , Record date : 2079-11-16 ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 SSN: 123-22-4567.","Medical Center , Main Street , Phone 555-555-0000 SSN: 555-555-0000."


# mode: mask, parameters: defaults

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("masked") \
    .setMode("mask") \



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.masked.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()


,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <DOCTOR> , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : <PATIENT> , MR # <MEDICALRECORD> Date : <DATE> ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : <DOCTOR> , <AGE> years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555","<HOSPITAL> , <STREET> , Phone <PHONE>"


all sensitive entities are masked with LABEL by default

There are 3 masking policies:


*   entity_labels - `default`
*   fixed_length_chars
*   same_length_char




## fixed_length_chars

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("masked") \
    .setMode("mask") \
    .setMaskingPolicy("fixed_length_chars") \
    .setFixedMaskLength(5) \

nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.masked.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()


,sentence,obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : ***** , ***** , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : ***** , MR # ***** Date : ***** ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : ***** , ***** years-old , Record date : ***** ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555","***** , ***** , Phone *****"


## same_length_chars

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("masked") \
    .setMode("mask") \
    .setMaskingPolicy("same_length_chars") \



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.masked.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("masked")).toPandas()


,sentence,masked
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : [********] , [********] , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : [*************] , MR # [*****] Date : [******] ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : [******] , ** years-old , Record date : [********] ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555","[***************************] , [***************] , Phone [*********]"


# multimode:
to deidentify entities with masking or fake data, we are able to use multimode options as following

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("masked") \
    .setMode("mask") \
    .setObfuscateDate(True) \
    .setDays(7)\
    .setAgeRanges([1,4,10,20,40,60,80,100])\
    .setSelectiveObfuscationModes({"OBFUSCATE": ["Date","Street","Doctor", "Patient","Age"],
                                    "mask_same_length_chars": ["MEDICALRECORD", "Phone"],
                                    "mask_entity_labels": ["HOSPITAL"],
                                    }) \



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.masked.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("masked")).toPandas()


,sentence,masked
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-20 , Rhina Brackett , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Myles Rosenthal , MR # [*****] Date : 01/20/93 ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : Octavio Manns , 51 years-old , Record date : 2079-11-16 ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555","<HOSPITAL> , 2525 S Downing Rd,3Rd Floor , Phone [*********]"


to obfuscate DATEs:
we put these two paramters
* `.setObfuscateDate(True) \`
* `.setDays(7)\`
and put into dictionary below

for other entities, we add this dictionary inside `.setSelectiveObfuscationModes() parameter.


    `{"OBFUSCATE": ["Date", "Street","Doctor", "Patient","Age"],
      "mask_same_length_chars": ["MEDICALRECORD", "Phone"],
      "mask_entity_labels": ["HOSPITAL"],
      }`

Also `age` is in "OBFUSCATE" list and .setAgeRanges() is declared, so age will be fake but reasonably according to related range: 55 will be between 40-60

In [ ]:
light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("masked") \
    .setMode("mask") \
    .setObfuscateDate(True) \
    .setDays(7)\
    .setRegion("us")\
    .setUnnormalizedDateMode("skip") \
    .setAgeRanges([1,4,10,20,40,60,80,100])\
    .setSelectiveObfuscationModes({"OBFUSCATE": ["Date","Street","Doctor", "Patient","Age"],
                                    "mask_same_length_chars": ["MEDICALRECORD", "Phone"],
                                    "mask_entity_labels": ["HOSPITAL"],
                                    }) \



nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

text ='''
Record date : 2093-01-13 , David Hale , M.D . ,
Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .
PCP : Oliveira , 55 years-old , Record date : 2079-11-09 . Discharged date : April 9 2024.
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555
'''

result = model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.select(F.explode(F.arrays_zip(result.sentence.result, result.masked.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("masked")).toPandas()


,sentence,masked
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-20 , Cruz Condon , M.D ."
1,", \nName : Hendrickson Ora , MR # 7194334 Date : 01/13/93 .",", \nName : Jeronimo Greaves , MR # [*****] Date : 01/20/93 ."
2,"PCP : Oliveira , 55 years-old , Record date : 2079-11-09 .","PCP : Corky Sox , 40 years-old , Record date : 2079-11-16 ."
3,Discharged date : April 9 2024.,Discharged date : April 9 2024.
4,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555","<HOSPITAL> , 4901 College Boulevard , Phone [*********]"


 note: date format of  `April 9 2024` chunk is not recognized  and
   ` .setUnnormalizedDateMode("skip") \`
parameter takes effect  Options: [mask, obfuscate, skip]. Default: obfuscate.

In [ ]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022',
               'Mark White was discharged on 03/01/2020',
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['10', '-2', '30', '-8']
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|10       |
|A002     |Mark White was discharged on 03/01/2020 |-2       |
|A003     |John was discharged on 03/15/2022       |30       |
|A004     |John Moore was discharged on 12/31/2022 |-8       |
+---------+----------------------------------------+---------+



# shiftdays

we can obfuscate the dates not only randomly or add/substract a constant day, but also change per document:

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document2"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_subentity")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner_subentity"])\
    .setOutputCol("ner_chunk")

light_deidentification = LightDeIdentification() \
    .setInputCols(["ner_chunk", "sentence"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setObfuscateDate(True)\
    .setLanguage("en") \
    .setSeed(10) \
    .setUseShiftDays(True)\

nlpPipeline = Pipeline(stages=[
      documentAssembler,
      documentHasher,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter,
      light_deidentification
])

empty_data = spark.createDataFrame([["",""]]).toDF("text","dateshift")

model = nlpPipeline.fit(empty_data)

result = model.transform(my_input_df)
result.select(F.explode(F.arrays_zip(result.sentence.result, result.obfuscated.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("obfuscated")).toPandas()

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


,sentence,obfuscated
0,Chris Brown was discharged on 10/02/2022,Miki Kins was discharged on 10/03/2022
1,Mark White was discharged on 03/01/2020,Dyane Dustman was discharged on 31/01/2020
2,John was discharged on 03/15/2022,Derek Jack was discharged on 04/12/2022
3,John Moore was discharged on 12/31/2022,Marlane Hatcher was discharged on 01/28/2023
